In [37]:
# # Downloading Dataset from drive and extracting
# !pip install gdown
# import gdown
# file_id='1qCHQprYeAoyaS-jCaUM1mc9Kg2RjZfCW'
# gdown.download(
#     f"https://drive.google.com/uc?export=download&confirm=pbef&id={file_id}",
# )
# !unzip 'Dataset.zip'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
import random
import warnings
import numpy as np
import cv2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16


In [15]:
warnings.simplefilter(action='ignore', category=FutureWarning)

train_path = '../input/sign-detection-3300/Dataset/Train'
test_path = '../input/sign-detection-3300/Dataset/Test'
data_gen=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input,validation_split=0.2)
train_batches = data_gen.flow_from_directory(directory=train_path, target_size=(64,64), class_mode='categorical', batch_size=16,shuffle=True,subset='training')
validation_batches = data_gen.flow_from_directory(directory=train_path, target_size=(64,64), class_mode='categorical', batch_size=16,shuffle=True,subset='validation')
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(64,64), class_mode='categorical', batch_size=16
                                                                                                                           , shuffle=True)

imgs, labels = next(train_batches)


#Plotting the images...
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(30,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


plotImages(imgs)
print(imgs.shape)
print(labels)


In [5]:

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Flatten())


model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))
#model.add(Dropout(0.2))
model.add(Dense(128,activation ="relu"))
#model.add(Dropout(0.3))
model.add(Dense(10,activation ="softmax"))

In [16]:
base_model = VGG16(weights="imagenet", include_top=False,input_shape=(64,64,3))
base_model.trainable = False ## Not trainable weights

flatten_layer = tf.keras.layers.Flatten()
dense_layer_1 = tf.keras.layers.Dense(50, activation='relu')
dense_layer_2 = tf.keras.layers.Dense(20, activation='relu')
prediction_layer = tf.keras.layers.Dense(10, activation='softmax')


model = tf.keras.models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [17]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history2 = model.fit(train_batches, epochs=8,  validation_data = validation_batches,)#, checkpoint])
imgs, labels = next(train_batches) # For getting next batch of imgs...

imgs, labels = next(test_batches) # For getting next batch of imgs...
scores = model.evaluate(imgs, labels, verbose=0)
print(f'{model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')


#model.save('best_model_dataflair.h5')
model.save('VGG.h5')

print(history2.history)

imgs, labels = next(test_batches)





In [25]:
model = keras.models.load_model(r"VGG.h5")

scores = model.evaluate(imgs, labels, verbose=0)
print(f'{model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

model.summary()

scores #[loss, accuracy] on test data...
model.metrics_names


word_dict = {0: '1', 1: '10', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9'}

In [26]:
print(scores)

In [27]:
print(train_batches.class_indices)
word_dict = train_batches.class_indices
print(word_dict)


In [28]:
word_dict_1 = {j:i for i,j in word_dict.items()}
print(word_dict_1)

In [29]:
model = keras.models.load_model(r"VGG.h5")

scores = model.evaluate(imgs, labels, verbose=0)
print(f'{model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

model.summary()

scores #[loss, accuracy] on test data...
model.metrics_names




In [30]:
predictions = model.predict(imgs, verbose=0)
print("predictions on a small set of test data--")
print("")
print(word_dict_1)
for i in predictions:
    print(word_dict_1[np.argmax(i)], end='   ')
plotImages(imgs)
print('Actual labels')
for i in labels:
    print(word_dict_1[np.argmax(i)], end='   ')

# print(imgs.shape)

# history2.history